In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from  urllib.parse import quote

def make_soup(resp: requests.Response):
    return BeautifulSoup(resp.content, 'html.parser')

In [5]:
keywords=["evga", "3090", "FTW"]
url = f"https://shop.hardware.fr/search/+ftxt-{quote('-'.join(keywords))}/"

resp = requests.get(url)
resp.raise_for_status()

print(resp.headers.get('content-type'))
try:
    json = resp.json()
except Exception as e:
    print(type(e))

bs = make_soup(resp)

text/html; charset=UTF-8
<class 'json.decoder.JSONDecodeError'>


In [87]:
items = bs.select("li[data-ref]")
print(len(items))

3


In [90]:
import re
item = items[0]
print(item.find(class_="description").get_text())
item_id = item.attrs["id"]
script_data = ''.join(s.string for s in bs.find_all("script", attrs={"src": None}))
re.search("#{id}.*?stock-wrapper.*?stock-([0-9])".format(id=item_id), script_data)[1]


EVGA GeForce RTX 3090 FTW3 GAMING
24 Go GDDR6X - HDMI/Tri DisplayPort - PCI Express (NVIDIA GeForce RTX 3090)


Call of Duty : Black Ops Cold War offert avec NVIDIA
                    




'9'

In [ ]:
target_url = "https://www.materiel.net/product-listing/stock-price/"
headers = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'x-requested-with': 'XMLHttpRequest'
}
payload = "json=%7B%22currencyISOCode3%22%3A%22EUR%22%2C%22offers%22%3A%5B%7B%22offerId%22%3A%22AR202009090100%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090101%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090099%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009100088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070099%22%2C%22marketplace%22%3Afalse%7D%5D%2C%22shops%22%3A%5B%7B%22shopId%22%3A-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202009090100%22%3A%220%22%2C%22AR202009090101%22%3A%220%22%2C%22AR202012070098%22%3A%220%22%2C%22AR202009090098%22%3A%220%22%2C%22AR202009090099%22%3A%220%22%2C%22AR202009100088%22%3A%220%22%2C%22AR202012070099%22%3A%220%22%7D"
# payload = "json=%7B%22currencyISOCode3%22%3A%22EUR%22%2C%22offers%22%3A%5B%7B%22offerId%22%3A%22AR202012010033%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012010032%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202001100026%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201912120087%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202001100027%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911180073%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911260039%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911260040%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011120046%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011120043%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011250037%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011250036%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012030019%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201705090271%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201802020021%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201802020020%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201708250043%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201811290081%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201705090270%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201802020019%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911270006%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201801230058%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201904110023%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009250061%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202002270105%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201906130097%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911190022%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202007080112%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202007080111%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201906280025%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201903040009%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201910170046%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202006080089%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201910170047%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202006080088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201911260088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202006160037%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011060098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012210010%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012210009%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011040097%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011040047%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011060096%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011040046%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009220022%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009220023%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009170013%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009170012%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201704070124%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010160068%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010200077%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009220060%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201901290073%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009290085%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011200137%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201910180176%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011090108%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201910080073%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010220025%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202005200108%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202011180025%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009140110%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012210015%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012230038%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012230037%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012220093%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012170057%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012110104%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012160052%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012160050%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012160051%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012160047%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012160046%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012140020%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012110051%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201610200151%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201601290073%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201903130089%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201910170079%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201902140058%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201908020049%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009070053%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010010052%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010300049%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201902140083%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR201903130088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202010050072%22%2C%22marketplace%22%3Afalse%7D%5D%2C%22shops%22%3A%5B%7B%22shopId%22%3A-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202006160037%22%3A%220%22%2C%22AR202012210009%22%3A%220%22%2C%22AR202012210010%22%3A%220%22%2C%22AR201802020021%22%3A%220%22%2C%22AR202012010033%22%3A%220%22%2C%22AR202012010032%22%3A%220%22%2C%22AR202001100026%22%3A%220%22%2C%22AR201912120087%22%3A%220%22%2C%22AR202001100027%22%3A%220%22%2C%22AR201911180073%22%3A%220%22%2C%22AR201911260039%22%3A%220%22%2C%22AR201911260040%22%3A%220%22%2C%22AR202011120046%22%3A%220%22%2C%22AR202011120043%22%3A%220%22%2C%22AR202011250037%22%3A%220%22%2C%22AR202011250036%22%3A%220%22%2C%22AR202012030019%22%3A%220%22%2C%22AR201705090271%22%3A%222%22%2C%22AR201802020020%22%3A%220%22%2C%22AR201708250043%22%3A%220%22%2C%22AR201811290081%22%3A%220%22%2C%22AR201705090270%22%3A%220%22%2C%22AR201802020019%22%3A%220%22%2C%22AR201911270006%22%3A%220%22%2C%22AR201801230058%22%3A%220%22%2C%22AR201904110023%22%3A%220%22%2C%22AR202009250061%22%3A%220%22%2C%22AR202002270105%22%3A%220%22%2C%22AR201906130097%22%3A%220%22%2C%22AR201911190022%22%3A%220%22%2C%22AR202007080112%22%3A%220%22%2C%22AR202007080111%22%3A%220%22%2C%22AR201906280025%22%3A%220%22%2C%22AR201903040009%22%3A%220%22%2C%22AR201910170046%22%3A%220%22%2C%22AR202006080089%22%3A%220%22%2C%22AR201910170047%22%3A%220%22%2C%22AR202006080088%22%3A%220%22%2C%22AR201911260088%22%3A%220%22%2C%22AR202011060098%22%3A%220%22%2C%22AR202011040097%22%3A%220%22%2C%22AR202011040047%22%3A%220%22%2C%22AR202011060096%22%3A%220%22%2C%22AR202011040046%22%3A%220%22%2C%22AR202009220022%22%3A%220%22%2C%22AR202009220023%22%3A%220%22%2C%22AR202009170013%22%3A%220%22%2C%22AR202009170012%22%3A%220%22%7D"
resp = requests.post(target_url, data=payload, headers=headers)

print(dict(resp.request.headers))
resp.raise_for_status()
json = resp.json()

In [104]:
import re
len(json["price"])
def is_in_stock(art):
    match = re.search(r"o-availability__value--stock_([0-9])", art)
    assert match
    return int(match[1]) <= 2

any(map(is_in_stock, json["stock"].values()))

False